<a href="https://colab.research.google.com/github/SangamSilwal/DeepLearning_DSeries/blob/main/T3_pytorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [32]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [33]:
df.drop(columns=["id","Unnamed: 32"],inplace=True)

In [34]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [35]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.84537207,  0.12820577, -0.8795569 , ..., -1.3143849 ,
        -0.5498205 , -0.4303407 ],
       [-0.26904132, -0.50713333, -0.33030417, ..., -0.27070754,
        -0.62878191, -0.42479246],
       [ 1.79829437,  0.49674767,  1.65736893, ...,  0.4567871 ,
        -1.04371637, -0.69998497],
       ...,
       [-0.97153402,  1.11852081, -0.97945872, ..., -1.24852621,
         0.2274075 , -0.42312799],
       [-0.61311938, -1.31656891, -0.60886858, ..., -0.61905735,
        -0.2030096 , -0.18677314],
       [ 0.43058408, -1.20578024,  0.39305131, ...,  0.34854712,
        -0.29280885, -0.7310551 ]])

In [36]:
y_train

,diagnosis
522,B
224,B
237,M
383,B
44,M
...,...
140,B
156,M
549,B
529,B


In [38]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [39]:
# Numpy array to pytorch tensor
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)
X_train_tensor.shape

torch.Size([455, 30])

In [51]:
from torch.nn.functional import binary_cross_entropy

In [66]:
# Defining the model

class MySimpleNN():

  def __init__(self,X):
    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z = torch.matmul(X,self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss


In [72]:
learning_rate = 0.1
epochs = 100

In [73]:
# Training Pipeline

model = MySimpleNN(X_train_tensor)

for epoch in range(epochs):
  y_pred = model.forward(X_train_tensor)
  loss = model.loss_function(y_pred,y_train_tensor)
  loss.backward()

  with torch.no_grad():
    model.weights -= learning_rate*model.weights.grad
    model.bias -= learning_rate*model.bias.grad

  # zero gradient
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch: {epoch+1}, Loss: {loss.item()}')


Epoch: 1, Loss: 3.6533673469070718
Epoch: 2, Loss: 3.5245074556890517
Epoch: 3, Loss: 3.394175160145805
Epoch: 4, Loss: 3.2554751078891937
Epoch: 5, Loss: 3.1122293671688586
Epoch: 6, Loss: 2.964330944189383
Epoch: 7, Loss: 2.810608818390204
Epoch: 8, Loss: 2.656927820738899
Epoch: 9, Loss: 2.4978782992195656
Epoch: 10, Loss: 2.3318852893521407
Epoch: 11, Loss: 2.160130917537178
Epoch: 12, Loss: 1.9933336745608208
Epoch: 13, Loss: 1.8257361716810445
Epoch: 14, Loss: 1.6608396208840446
Epoch: 15, Loss: 1.5044480166418486
Epoch: 16, Loss: 1.358381600461419
Epoch: 17, Loss: 1.2274225791930802
Epoch: 18, Loss: 1.1136771079537675
Epoch: 19, Loss: 1.0188330167473585
Epoch: 20, Loss: 0.9436204824246452
Epoch: 21, Loss: 0.8871967194644007
Epoch: 22, Loss: 0.8468795933706115
Epoch: 23, Loss: 0.8187585328007528
Epoch: 24, Loss: 0.7989373607006202
Epoch: 25, Loss: 0.7844208725815834
Epoch: 26, Loss: 0.773237499340771
Epoch: 27, Loss: 0.7641827321486648
Epoch: 28, Loss: 0.7565416173030668
Epoch: 2

In [74]:
model.bias

tensor([-0.4650], dtype=torch.float64, requires_grad=True)

In [75]:
model.weights

tensor([[ 0.3410],
        [-0.0229],
        [ 0.1005],
        [-0.2332],
        [ 0.0182],
        [-0.2804],
        [ 0.2003],
        [-0.3078],
        [-0.0991],
        [ 0.0518],
        [-0.0189],
        [ 0.1513],
        [-0.0163],
        [-0.2814],
        [-0.0371],
        [ 0.4333],
        [-0.1290],
        [ 0.1471],
        [ 0.0219],
        [-0.1903],
        [ 0.1902],
        [-0.1076],
        [-0.0014],
        [ 0.3277],
        [ 0.2136],
        [-0.1867],
        [-0.1658],
        [-0.1897],
        [ 0.1342],
        [ 0.2106]], dtype=torch.float64, requires_grad=True)

In [78]:
# model evaluation

with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred >0.5).float()

In [79]:
y_pred

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      